# Transfer Learning

https://www.youtube.com/watch?v=c36lUUr864M

In [1]:
# importing necessary libraries
import numpy as np

import torch as tr
import torch.nn as nn
# import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms
# import torchvision.transforms as transforms

import matplotlib.pyplot as plt

import time 
import os
import copy


In [2]:
# device configuration
device = tr.device('cuda' if tr.cuda.is_available() else 'cpu')

In [3]:
mean = np.array([0.485,0.456, 0.406])
std = np.array([0.229,0.224,0.225])

In [4]:
data_transforms = {
    'train' : transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),

    'val' : transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
} 

In [5]:
# import data 
data_dir = './hymenoptera_data'
sets = ['train', 'val']

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}

dataLoaders = {x: tr.utils.data.DataLoader(image_datasets[x], batch_size= 4, shuffle=True, num_workers=0) for x in ['train', 'val']}

In [6]:
dataset_size = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

print(class_names)

['ants', 'bees']


In [20]:
def train_model(model, criterian, optimizer, scheduler, num_epochs = 25):
    since  =time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-'*10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() # set model to training mode
            else:
                model.eval() # set model to evaluation mode

            running_loss = 0.0
            running_corrects = 0

            # itrate over data 
            for input_i, label_s in dataLoaders[phase]:
                input_i = input_i.to(device)
                label_s = label_s.to(device)

                # forward
                # track history if only in train
                with tr.set_grad_enabled(phase == 'train'):
                    outputs = model(input_i)
                    _, pred = tr.max(outputs, 1)
                    loss = criterian(outputs, label_s)

                    # backward + optimizer only if in training phase

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                
                # statistics
                running_loss += loss.item() * input_i.size(0)
                running_corrects += tr.sum(pred == label_s.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss/ dataset_size[phase]
            epoch_acc = running_corrects.double() / dataset_size[phase]

            print(f'{phase} --> Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.2f}s')
    print(f'Best val accuracy: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)

    return model
    


In [21]:
model = models.resnet18(pretrained = True)

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
model.to(device)

criteria = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

# scheduler

step_lr_schedular = lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

model = train_model(model, criteria, optimizer, step_lr_schedular, num_epochs= 2)



Epoch 0/1
----------
train --> Loss: 0.6676 Acc: 0.6311
val --> Loss: 0.5304 Acc: 0.7451

Epoch 1/1
----------
train --> Loss: 0.5236 Acc: 0.7459
val --> Loss: 0.3785 Acc: 0.8954

Training complete in 1m 4.99s
Best val accuracy: 0.8954


In [22]:
model = models.resnet18(pretrained = True)

for param in model.parameters():
    param.requires_grad = False


num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
model.to(device)

criteria = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001)

# scheduler

step_lr_schedular = lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

model = train_model(model, criteria, optimizer, step_lr_schedular, num_epochs= 2)

Epoch 0/1
----------
train --> Loss: 0.6467 Acc: 0.5738
val --> Loss: 0.5750 Acc: 0.6993

Epoch 1/1
----------
train --> Loss: 0.5404 Acc: 0.7336
val --> Loss: 0.4345 Acc: 0.8497

Training complete in 0m 29.99s
Best val accuracy: 0.8497


In [ ]:
# 3:50:02